### Detect Data Drift in ML Models
**Objective**: Monitor and detect changes in data distributions that impact ML model performance.

**Task**: Categorical Feature Drift

**Steps**:
1. Load the baseline distribution for a categorical feature (e.g., gender ) from your training dataset.
2. Load the same feature from your current production data.
3. Use chi-squared tests to compare the distributions of the categorical feature.
4. Step 4: If significant drift is detected, investigate the cause and update the model as needed.

In [1]:
# write your code from here
import pandas as pd
from scipy.stats import chi2_contingency

# === Step 1: Load Baseline and Production Data ===

def load_baseline_data():
    return pd.DataFrame({
        'gender': ['male', 'female', 'male', 'female', 'female', 'male', 'female']
    })

def load_production_data():
    return pd.DataFrame({
        'gender': ['male', 'male', 'male', 'female', 'male', 'male', 'female']
    })

# === Step 2: Validate Input ===

def validate_categorical_column(df, column_name):
    if df.empty:
        raise ValueError("DataFrame is empty.")
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame.")
    if df[column_name].isnull().any():
        raise ValueError(f"Column '{column_name}' contains null values.")

# === Step 3: Chi-Squared Drift Detection ===

def detect_categorical_drift(base_df, prod_df, column_name, significance_level=0.05):
    base_counts = base_df[column_name].value_counts().sort_index()
    prod_counts = prod_df[column_name].value_counts().sort_index()

    # Align the categories
    all_categories = sorted(set(base_counts.index).union(set(prod_counts.index)))
    base_freq = [base_counts.get(cat, 0) for cat in all_categories]
    prod_freq = [prod_counts.get(cat, 0) for cat in all_categories]

    contingency_table = [base_freq, prod_freq]
    chi2, p_value, _, _ = chi2_contingency(contingency_table)

    drift_detected = p_value < significance_level
    return {
        "chi2_statistic": chi2,
        "p_value": p_value,
        "drift_detected": drift_detected,
        "categories": all_categories,
        "baseline_distribution": base_freq,
        "production_distribution": prod_freq
    }

# === Step 4: Run Detection ===

try:
    baseline_df = load_baseline_data()
    production_df = load_production_data()
    column_to_check = 'gender'

    # Validate inputs
    validate_categorical_column(baseline_df, column_to_check)
    validate_categorical_column(production_df, column_to_check)

    # Detect Drift
    result = detect_categorical_drift(baseline_df, production_df, column_to_check)

    print("✅ Drift Detection Result:")
    print(f"Categories: {result['categories']}")
    print(f"Baseline: {result['baseline_distribution']}")
    print(f"Production: {result['production_distribution']}")
    print(f"Chi-squared Statistic: {result['chi2_statistic']:.4f}")
    print(f"P-value: {result['p_value']:.4f}")
    print("🚨 Drift Detected!" if result['drift_detected'] else "✅ No significant drift detected.")

except Exception as e:
    print(f"❌ Error: {e}")


✅ Drift Detection Result:
Categories: ['female', 'male']
Baseline: [4, 3]
Production: [2, 5]
Chi-squared Statistic: 0.2917
P-value: 0.5892
✅ No significant drift detected.


In [2]:
import unittest

class TestCategoricalDrift(unittest.TestCase):

    def test_valid_drift_detection(self):
        base_df = pd.DataFrame({'gender': ['male'] * 3 + ['female'] * 4})
        prod_df = pd.DataFrame({'gender': ['male'] * 6 + ['female'] * 1})
        result = detect_categorical_drift(base_df, prod_df, 'gender')
        self.assertIn('chi2_statistic', result)
        self.assertIn('p_value', result)
        self.assertIn('drift_detected', result)

    def test_no_drift_detection(self):
        base_df = pd.DataFrame({'gender': ['male', 'female'] * 5})
        prod_df = pd.DataFrame({'gender': ['male', 'female'] * 5})
        result = detect_categorical_drift(base_df, prod_df, 'gender')
        self.assertFalse(result['drift_detected'])

    def test_missing_column(self):
        df = pd.DataFrame({'age': [1, 2, 3]})
        with self.assertRaises(ValueError):
            validate_categorical_column(df, 'gender')

    def test_null_values(self):
        df = pd.DataFrame({'gender': ['male', None, 'female']})
        with self.assertRaises(ValueError):
            validate_categorical_column(df, 'gender')

    def test_empty_dataframe(self):
        df = pd.DataFrame(columns=['gender'])
        with self.assertRaises(ValueError):
            validate_categorical_column(df, 'gender')

if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)


.....
----------------------------------------------------------------------
Ran 5 tests in 0.010s

OK
